In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import geopandas as gpd

from shapely.wkt import loads
from shapely.geometry import Polygon, MultiPolygon

from shapely import wkt

import os

In [30]:
files = {'marine_protected_areas': ('../../data/PR100/Habitat/marine_protected_areas.parquet',
  'EPSG:32161',
  'centroid'),
 'land_nationwide_hapc': ('../../data/PR100/Habitat/land_nationwide_hapc.parquet',
  'EPSG:32161',
  'centroid'),
 'artificial_reefs': ('../../data/PR100/Habitat/artificial_reefs.parquet',
  'EPSG:32161',
  'irrelevant'),
 'nationwide_hapc': ('../../data/PR100/Habitat/nationwide_hapc.parquet',
  'EPSG:32161',
  'irrelevant'),
 'areas_protegidas_agencia': ('../../data/PR100/Habitat/areas_protegidas_agencia.parquet',
  'EPSG:32161',
  'centroid'),
 'noaa_charted_submarine_cables_30m_buffer': ('../../data/PR100/Infrastructure/noaa_charted_submarine_cables_30m_buffer.parquet',
  'EPSG:32161',
  'irrelevant'),
 'setbacks_xmission': ('../../data/PR100/Infrastructure/setbacks_xmission.parquet',
  'EPSG:32161',
  'lines'),
 'submarine_cable_30m_buffer': ('../../data/PR100/Infrastructure/submarine_cable_30m_buffer.parquet',
  'EPSG:32161',
  'irrelevant'),
 'pipeline_areas': ('../../data/PR100/Infrastructure/pipeline_areas.parquet',
  'EPSG:32161',
  'irrelevant'),
 'setbacks_roads': ('../../data/PR100/Infrastructure/setbacks_roads.parquet',
  'EPSG:4326',
  'density'),
 'setbacks_runway': ('../../data/PR100/Infrastructure/setbacks_runway.parquet',
  'EPSG:4326',
  'centroid'),
 'setbacks_airport': ('../../data/PR100/Infrastructure/setbacks_airport.parquet',
  'EPSG:4326',
  'points'),
 'setbacks_buildings': ('../../data/PR100/Infrastructure/setbacks_buildings.parquet',
  'EPSG:4326',
  'density'),
 'federal_and_state_waters': ('../../data/PR100/Boundaries/federal_and_state_waters.parquet',
  'EPSG:32161',
  'centroid'),
 'coastal_zone_mgmt_act_boundary': ('../../data/PR100/Boundaries/coastal_zone_mgmt_act_boundary.parquet',
  'EPSG:32161',
  'centroid'),
 'land_protected_areas': ('../../data/PR100/Boundaries/land_protected_areas.parquet',
  'EPSG:32161',
  'centroid'),
 'water_bodies': ('../../data/PR100/Topography/water_bodies.parquet',
  'EPSG:4326',
  'centroid'),
 'wrecks_and_obstructions': ('../../data/PR100/Hazards/wrecks_and_obstructions.parquet',
  'EPSG:4326',
  'points'),
 'ocean_disposal_sites': ('../../data/PR100/Hazards/ocean_disposal_sites.parquet',
  'EPSG:32161',
  'centroid'),
 'tropical_cyclone_storm_segments': ('../../data/PR100/Hazards/tropical_cyclone_storm_segments.parquet',
  'EPSG:32161',
  'lines'),
 'tropical_cyclone_wind_exposure': ('../../data/PR100/Hazards/tropical_cyclone_wind_exposure.parquet',
  'EPSG:32161',
  'centroid'),
 'unexploded_ordinance_areas_fuds': ('../../data/PR100/Hazards/unexploded_ordinance_areas_fuds.parquet',
  'EPSG:32161',
  'centroid'),
 'danger_zones_and_restricted_areas': ('../../data/PR100/Hazards/danger_zones_and_restricted_areas.parquet',
  'EPSG:32161',
  'centroid'),
 'unexploded_ordinance_areas': ('../../data/PR100/Hazards/unexploded_ordinance_areas.parquet',
  'EPSG:4326',
  'irrelevant')}

In [2]:
def find_parquet_files(directory):
    """
    Find all Parquet files within a directory and its subdirectories.

    :param directory: The base directory to search.
    :return: A list of paths to Parquet files.
    """
    parquet_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.parquet'):
                parquet_files.append(os.path.join(root, file))
    return parquet_files

In [13]:
def get_gdf(parquet_file, crs, grid):
    df = pd.read_parquet(parquet_file)
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    gdf.set_crs(crs, inplace=True)
    gdf.to_crs(grid.crs, inplace=True)
    return gdf

In [5]:
def create_grid(x_start, x_end, y_start, y_end, square_size):
    grid = []
    for x in np.arange(x_start, x_end, square_size):
        for y in np.arange(y_start, y_end, square_size):
            # Define the square's coordinates
            square = Polygon([(x, y), (x + square_size, y), (x + square_size, y + square_size), (x, y + square_size)])
            grid.append(square)
    return gpd.GeoDataFrame({'geometry': grid}, crs="EPSG:32161")

In [17]:
# def rasterize_polygons(grid, polygons):
#     centroids = grid.geometry.centroid
#     return centroids.apply(lambda x: polygons.contains(x).any())

In [22]:
def rasterize(grid, data):
    # Add a column to environment_gdf to keep track of original indices
    grid['original_index'] = grid.index
    
    # Perform a spatial join
    joined_gdf = gpd.sjoin(grid, data, how='right', predicate='intersects')
    
    # Create the binary mask column based on whether the original index appears in joined_gdf
    rasterized = grid['original_index'].isin(joined_gdf['original_index'])
    
    # Drop the 'original_index' column if no longer needed
    grid.drop(columns=['original_index'], inplace=True)

    return rasterized

In [23]:
def rasterize_density(grid, data):
    joined_gdf = gpd.sjoin(grid, data[['geometry']], how='left', op='intersects')
    count = joined_gdf.groupby('index').size()
    return count.reindex(grid.index, fill_value=0)

In [26]:
def process_parquet(file_path, crs, type, handle, grid):
    # Create a GeoDataFrame
    gdf = get_gdf(file_path, crs, grid)
    if type == 'irrelevant':
        pass
    elif type == 'density':
        gdf[handle] = rasterize_density(grid, gdf)
    else:
        gdf[handle] = rasterize(grid, gdf)

In [27]:
def process_all_parquets(files, grid):
    for handle, file_data in files.items():
        file_path = file_data[0]
        crs = file_data[1]
        type = file_data[2]
        print(f'Processing file: {file}')
        process_parquet(file_path, crs, type, hangle, grid)

In [28]:
parquet_files = find_parquet_files('../../data')

In [29]:
environment_gdf = create_grid(100000, 300000, 200000, 300000, 500)

In [ ]:
process_all_parquets(parquet_files, environment_gdf)